In [10]:
import sys
from SciExpeM_API.SciExpeM import SciExpeM
my_sciexpem = SciExpeM(username=, password=)

from SciExpeM_API.Models import *

Attention. SciExpeM is a singleton.


### Common properties

- name
- units
- value
- source_type

In [72]:
P = 12 #atm
phi = 0.25 #unitless
# c3 = CommonProperty(name='temperature', units='K', value='1098', source_type='reported')
c1 = CommonProperty(name='pressure', units='atm', value=str(P), source_type='reported')
#c2 = CommonProperty(name='residence time', units='s', value='0.004', source_type='reported')
# if parametric analysis, report only the common feature, see other examples
commonprop = [c1,]

### Initial Species

- name
- units
- value
- source_type
- configuration

In [73]:
# THIS REFERS TO THE SIMULATION
# PREMIXED IS DEFAULT AND MUST BE INDICATED UNLESS IT'S A CF FLAME
species = ['C7H8', 'O2', 'N2']
composition = ['0.00580', '0.20887', '0.78533']
comp_unit = 'mole fraction'
srctype = 'reported'
config = 'premixed'
################# do not edit
inspecies = []
for i, s in enumerate(species):
    si = my_sciexpem.filterDatabase(model_name='Specie', preferredKey=s)[0]
    ii = InitialSpecie(name=species[i], 
                    specie=si, 
                    units=comp_unit, 
                    value=composition[i], 
                    source_type=srctype, 
                    configuration=config)

    inspecies.append(ii)

### Data columns

- name
- label
- units
- data
- dg_id 
- dg_label
- source_type


In [74]:
T = [1152,1169,1171,1191,1220,1230,1231,1263,1268,1291,1385,1400,]
IDT_MUS = [966,600,687,639,465,398,475,342,298,233,151,94,]

# DO NOT EDIT
d1 = DataColumn(name='temperature',
                units='K', 
                dg_id='dg1',
                dg_label='experimental_data', # qua è il tipo "regolare"
                data= T,
                source_type='reported')

# uncertainty deve avere la stessa unità di misura della colonna associata
# id ovviamente deve essere sempre dello stesso datagroup e anche il label
# devi mettere PRIMA l'incertezza e poi il dato!
#d2 = DataColumn(name = 'uncertainty',  
#                units = 'us', 
#                dg_id = 'dg1',
#                dg_label = 'experimental_data',
#                data = [0.1, 0.2, 0.3, 0.4, 0.5],
#                source_type = 'reported',
#                uncertainty_bound = 'plusminus',  # oppure percentage
#                uncertainty_kind = 'relative')
#
d2= DataColumn(name='ignition delay',
                units='us', 
                dg_id='dg1',
                dg_label='experimental_data',
                data = IDT_MUS,
                source_type = 'reported',)
                #uncertainty_reference=d2) # RIFERIMENTO A INCERTEZZA
datacols = [d1, d2]

### FilePaper
 - description*
 - reference_doi*
 - author*
 - title*
 - year*
 - volume
 - page
 - journal

In [36]:
# QUESTO PIù COMODO DA FARE DIRETTAMENTE SU SCIEXPEM E POI FAI COPIA INCOLLA QUI
file_paper = FilePaper(reference_doi="10.1016/J.PROCI.2008.05.004", 
                       author="Shen, Hsi-Ping S.; Vanderover, Jeremy; Oehlschlaeger, Matthew A.",                       
                       title="Probing The Antagonistic Effect Of Toluene As A Component In Surrogate Fuel Models At Low Temperatures And High Pressures. A Case Study Of Toluene/Dimethyl Ether Mixtures",
                       year="2009",
                       description="Shen, Hsi-Ping S.; Vanderover, Jeremy; Oehlschlaeger, Matthew A. - Proceedings Of The Combustion Institute, 2009, (32), 165-172",
                       )

### OpenSMOKE input file if available

In [75]:
input_path = r'D:\POLI\OPENSMOKE\TOLUENE\IDT\Shen_2009\12 atm\025\input.dic'
with open(input_path) as f:
    inputstr = f.read()
# NB è MEGLIO METTERE TUTTO SULLA STESSA RIGA!! QUINDI TIPO INPUT DEVE ESSERE COSì

### Assembly the experiment

In [76]:
e = Experiment(reactor='shock tube', 
               experiment_type='ignition delay measurement', 
               ignition_type='p-d/dt max',
               file_paper=file_paper, 
               data_columns= datacols,  #NON C'è L'INCERTEZZA NELLE DATA COLUMNS!!
               initial_species=inspecies, 
               common_properties=commonprop,
               os_input_file=inputstr,
               t_inf = min(T), t_sup = max(T), #  QUESTA è LA SEZIONE CHARACTERISTICS
               # PER LFS METTI INF = SUP COME LA T DI INGRESSO
               p_inf = P, p_sup = P,
               phi_inf = phi, phi_sup = phi,
               fuels = ['C7H8']) # LISTA DI FUEL DI INGRESSO

In [ ]:
e.serialize()
# QUESTO TI FA VEDERE IL JSON DELLE COSE CHE STAI CARICANDO 
# IN MODO DA CONTROLLARE CHE SIA TUTTO A POSTO

### Send Experiment

In [77]:
my_sciexpem.insertElement(e, verbose=True)
# QUI DOVRESTI RIUSCIRE A FARLO SMOOTHLY

Experiment element inserted successfully.
